<a href="https://colab.research.google.com/github/sgoodfriend/rl-algo-impls/blob/main/rl_algo_impls/microrts/colab_microrts_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running RoundRobinTournament in Google Colab
MicroRTS has to talk to a Python server that runs code from the [sgoodfrieng/rl-algo-impls](https://github.com/sgoodfriend/rl-algo-impls) repo. This trained a residual convolutional model using PPO with invalid action masking. [Farama-Foundation/MicroRTS](https://github.com/Farama-Foundation/MicroRTS) was the inspiration for the training; however, at this point the useful bits of code are in rl-algo-impls as I extended behavior slightly.

## Installation
1. Clone repo

In [1]:
%%capture
!git clone https://github.com/sgoodfriend/rl-algo-impls.git

2. Install dependencies and expose `rai_microrts` command-line endpoint. Ubuntu 20.04's default JDK is Java 11. setuptools and wheel had to be downgraded because rl-algo-impls depends on stable-baslines3, which depends on gym 0.21.

In [2]:
%%capture
!apt install -y default-jdk
!pip install setuptools==65.5.0
!pip install wheel==0.38.4

%cd /content/rl-algo-impls
!git checkout 2519ac50fb82ee9b48eea09cb7240f830c60a2e7
!python -m pip install -e .

3. Download model. This model was trained on a series of 

In [3]:
%%capture
%cd /content/rl-algo-impls/
%mkdir -p saved_models/ppo-Microrts-selfplay-dc-phases-A10-S1-best
%cd saved_models/ppo-Microrts-selfplay-dc-phases-A10-S1-best
!wget https://api.wandb.ai/files/sgoodfriend/rl-algo-impls-benchmarks/df4flrs4/ppo-Microrts-selfplay-dc-phases-A10-S1-best.zip
!unzip ppo-Microrts-selfplay-dc-phases-A10-S1-best.zip

In [ ]:
%cd /content/rl-algo-impls/rl_algo_impls/microrts/java
!java -cp "rai.jar:$(find lib -name "*.jar" | tr '\n' ':')" tournaments.RAIRoundRobinTournament

/content/rl-algo-impls/rl_algo_impls/microrts/java
tournaments.RoundRobinTournament: Starting tournament
MATCH UP: RAISocketAI vs RandomBiasedAI
preGameAnalysis player 1 took 691
preGameAnalysis player 1 took 0
Winner: 0  in 557 cycles
RAISocketAI : null
RandomBiasedAI : null
AI1 time usage, average:  51.64991023339318, # times over budget: 14 (avg 118.5) , # times over 2*budget: 0 (avg NaN)
AI2 time usage, average:  0.026929982046678635, # times over budget: 0 (avg NaN) , # times over 2*budget: 0 (avg NaN)
MATCH UP: RAISocketAI vs ContinuingAI(NaiveMCTS(100, -1, 100,10,0.3, 1.0, 0.0, 1.0, 0.4, 1.0, RandomBiasedAI, SimpleSqrtEvaluationFunction3))
preGameAnalysis player 1 took 720
preGameAnalysis player 1 took 0
Winner: 0  in 650 cycles
RAISocketAI : null
ContinuingAI(NaiveMCTS(100, -1, 100,10,0.3, 1.0, 0.0, 1.0, 0.4, 1.0, RandomBiasedAI, SimpleSqrtEvaluationFunction3)) : Total runs: 167554, runs per action: 2094.425, runs per cycle: 417.8404, average time per cycle: 100.341644, max bra